In [6]:
from openai import OpenAI
import os
import pandas as pd

In [3]:
api_key = os.environ.get('api_key')

In [9]:
df = pd.read_csv('sales_data_sample.csv')

In [13]:
df.groupby("QTR_ID").sum()['SALES']


QTR_ID
1    2350817.73
2    2048120.30
3    1758910.81
4    3874780.01
Name: SALES, dtype: float64

In [ ]:
#SELECT SUM(SALES) FROM sales_data_sample 
# GROUP BY QTR_ID